<a href="https://colab.research.google.com/github/katarinagresova/M7DataSP_2020/blob/main/assignment_08/G_quadruplexes_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook was created for classification of G-quadruplexes, which was part of [assignment 08](https://github.com/simecek/dspracticum2020/tree/master/lecture_08/assignment) in subject [Data Science Practicum 2020](https://github.com/simecek/dspracticum2020). Most of the code was taken from [G4_benchmark notebook](https://github.com/simecek/dspracticum2020/blob/master/lecture_08/assignment/g4/G4_benchmark.ipynb).

Several architectures were tried, but the best results were from model created by concatenating cnn model and lstm model.

#Setup

In [1]:
!pip install biopython

In [2]:
import urllib.request
from pathlib import Path
from Bio import SeqIO
import numpy as np
import gzip

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

#Reshaping data from fasta to txt


In [3]:
classes = ['notpresent', 'present']
sets = ['train', 'valid']

for c in classes:
    for s in sets:
        urllib.request.urlretrieve(f"https://github.com/simecek/dspracticum2020/raw/master/lecture_08/assignment/g4/g4_{c}_{s}.fa.gz", f"g4_{c}_{s}.fa.gz")

In [4]:
for c in classes:
    for s in sets:
        Path(f"data/{s}/{c}").mkdir(parents=True, exist_ok=True)

In [ ]:
counter = {}
for c in classes:
    counter[c] = 0
    for s in sets:
        with gzip.open(f"g4_{c}_{s}.fa.gz", "rt") as handle:
            for record in SeqIO.parse(handle, "fasta"):
                counter[c] += 1
                id = record.id
                with open(f"data/{s}/{c}/{id}.txt", "w") as fw:
                    fw.writelines([" ".join(str(record.seq))])

for s in counter:
  print(f"Found {counter[s]} records of class {s}")

So we are working with balanced dataset.

#Reading data

In [ ]:
batch_size = 256

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'data/train/',
    batch_size=batch_size,
    class_names=classes)

In [ ]:
raw_valid_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'data/valid/',
    batch_size=batch_size,
    class_names=classes)

In [ ]:
vectorize_layer = TextVectorization(output_mode='int')

train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)
vectorize_layer.set_vocabulary(vocab=np.asarray(['a', 'c', 't', 'g', 'n']))

In [ ]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = raw_train_ds.map(vectorize_text)
valid_ds = raw_valid_ds.map(vectorize_text)

#Model training

In [ ]:
# one-hot encoding
onehot_layer = keras.layers.Lambda(lambda x: tf.one_hot(tf.cast(x,'int64'), 4))

In [ ]:
model_cnn_lstm = tf.keras.Sequential([
    onehot_layer,
    keras.layers.Conv1D(32, kernel_size=6, data_format='channels_last', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling1D(),
    keras.layers.Conv1D(16, kernel_size=6, data_format='channels_last', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling1D(),
    keras.layers.Conv1D(4, kernel_size=6, data_format='channels_last', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling1D(),
    keras.layers.Dropout(0.3),
    keras.layers.LSTM(128, return_sequences=True),
    keras.layers.LSTM(128, return_sequences=False),
    keras.layers.Dense(1, activation="sigmoid")
])

model_cnn_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
import matplotlib.pyplot as plt
def plot_accuracy(history):
  plt.plot(history.history['accuracy'], label='accuracy')
  plt.plot(history.history['val_accuracy'], label='val_accuracy')
  plt.ylim([0.9, 1])
  plt.xlabel('Epoch')
  plt.ylabel('Error')
  plt.legend()
  plt.grid(True)

In [ ]:
epochs = 10
history = model_cnn_lstm.fit(
    train_ds,
    epochs=epochs,
    validation_data = valid_ds)

In [ ]:
plot_accuracy(history)